In [1]:
import sys
sys.path.append('..')

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import cupy as _cu
from tqdm.auto import tqdm
import numpy as np
import math

In [4]:
import scaredcu

In [5]:
import estraces

In [6]:
ths = estraces.read_ths_from_ets_file(f'kyber_basemult_montgomery_pon_t0.ets')

In [7]:
import scaredcu.gpu_format as gpu_format

ths_ = gpu_format.read_ths_from_ram(_cu.array(ths.samples)[:,:], c=_cu.array(ths.c)[:,:],
                                      s=_cu.array(ths.s)[:,:])

In [8]:
## from kyber_reverse.ipynb
REDUCTION = 'MONTGOMERY'
if REDUCTION == 'MONTGOMERY':

    S0L, S0R, S1L, S1R = 93, 9053, 9189, 18149
    C0L, C0R, C1L, C1R = 76, 80, 128, 132
    
    K = 140
    X = 4

elif REDUCTION == 'PLANTARD':

    S0L, S0R, S1L, S1R = 128, 9600, 9740, 19212
    C0L, C0R, C1L, C1R = 53, 57, 101, 105
    
    K = 148
    X = 0


def make_frame(key_index, d, is_high=True):
    x = 0 if is_high else X    
    T = S0L if d == 0 else S1L
    if key_index % 2 == 0:
        return range(T + K*(key_index//2) + C0L + x, T + K*(key_index//2) + C0R + x)
    else:
        return range(T + K*(key_index//2) + C1L + x, T + K*(key_index//2) + C1R + x)


In [9]:
def print_scores(ki, scores, guesses, scores_sorted=None, l=10):
    if scores_sorted is None:
        scores_sorted = scores.argsort()[::-1]

    for i in range(l):
        print(ki, i, guesses[scores_sorted[i]], scores[scores_sorted[i]])
    print()

In [10]:
step = 900
batch_size = 450
key_index = 0
scaredcu.container.set_batch_size(batch_size)

model = scaredcu.models.AbsoluteValue()
selection_function = scaredcu.lbc.kyber.selection_functions.BaseMulMontyIterated(cu_step=1<<19, words=[key_index],
                                                                                 low=False, high=True, neg_trick=True)

In [11]:
selection_function.reset()

for _ in tqdm(range(selection_function.num_steps)):

    assert selection_function.done() == False


    cpa = scaredcu.CPAAttackAlternative(selection_function=selection_function,
                                            model=model,
                                            discriminant=scaredcu.maxabs,
                                            convergence_step=None)


    mean = ths_.samples.mean(axis=0)
    pp_ = scaredcu.preprocesses.high_order.CenteredProduct(mean=mean, frame_1=make_frame(key_index, 0, True),
                                                            frame_2=make_frame(key_index, 1, True), mode='same')

    for i in range(0, 900, step):
        container = scaredcu.Container(ths_[i:i + step], preprocesses=[pp_])
        cpa.run(container)
    selection_function.save_scores(cpa.scores)
    print(cpa.scores.shape, cpa.scores.argmax(), selection_function.guesses[cpa.scores.argmax()], selection_function.scores.shape)
    print_scores(key_index, selection_function.scores[:,0], selection_function.np_guesses)
    selection_function.next()

  0%|          | 0/11 [00:00<?, ?it/s]

AttributeError: 'ndarray' object has no attribute '_basemul_low'

In [12]:
cpa

In [1]:
for x in [21932846, -21932846, 3562152210, -3562152210, 752167598, -752167598, 3417653460, -3417653460, 2112004045, -2112004045, 932791035, -932791035, 2951903026, -2951903026, 1419184148, -1419184148, 1817845876, -1817845876, 3434425636, -3434425636, 4233039261, -4233039261, 300609006, -300609006, 975366560, -975366560, 2781600929, -2781600929, 3889854731, -3889854731, 3935010590, -3935010590, 2197155094, -2197155094, 2130066389, -2130066389, 3598276897, -3598276897, 2308109491, -2308109491, 2382939200, -2382939200, 1228239371, -1228239371, 1884934581, -1884934581, 3466679822, -3466679822, 1211467195, -1211467195, 2977706375, -2977706375, 3144137970, -3144137970, 3080919767, -3080919767, 945692709, -945692709, 3015121229, -3015121229, 345764865, -345764865, 826997308, -826997308, 2043625172, -2043625172, 2964804700, -2964804700, 2628071007, -2628071007, 4154339049, -4154339049, 483812778, -483812778, 3288636719, -3288636719, 2696449880, -2696449880, 2122325384, -2122325384, 1371447954, -1371447954, 411563403, -411563403, 3577634219, -3577634219, 976656727, -976656727, 2708061387, -2708061387, 723783916, -723783916, 3181552825, -3181552825, 3346694253, -3346694253, 3617629408, -3617629408, 1408862808, -1408862808, 519937465, -519937465, 1323711759, -1323711759, 1474661346, -1474661346, 2773859924, -2773859924, 3580214553, -3580214553, 1143088323, -1143088323, 2221668274, -2221668274, 1563682897, -1563682897, 2417773720, -2417773720, 1327582262, -1327582262, 2722253228, -2722253228, 3786641338, -3786641338, 1141798155, -1141798155, 2779020594, -2779020594]:
    print(x & 0xffffffff, end=', ')
print()

21932846, 4273034450, 3562152210, 732815086, 752167598, 3542799698, 3417653460, 877313836, 2112004045, 2182963251, 932791035, 3362176261, 2951903026, 1343064270, 1419184148, 2875783148, 1817845876, 2477121420, 3434425636, 860541660, 4233039261, 61928035, 300609006, 3994358290, 975366560, 3319600736, 2781600929, 1513366367, 3889854731, 405112565, 3935010590, 359956706, 2197155094, 2097812202, 2130066389, 2164900907, 3598276897, 696690399, 2308109491, 1986857805, 2382939200, 1912028096, 1228239371, 3066727925, 1884934581, 2410032715, 3466679822, 828287474, 1211467195, 3083500101, 2977706375, 1317260921, 3144137970, 1150829326, 3080919767, 1214047529, 945692709, 3349274587, 3015121229, 1279846067, 345764865, 3949202431, 826997308, 3467969988, 2043625172, 2251342124, 2964804700, 1330162596, 2628071007, 1666896289, 4154339049, 140628247, 483812778, 3811154518, 3288636719, 1006330577, 2696449880, 1598517416, 2122325384, 2172641912, 1371447954, 2923519342, 411563403, 3883403893, 3577634219, 7